# Immune exclusion signature scoring and T cell quantification
- Correspond to Figure R1-4

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
import seaborn as sn

In [ ]:
import anndata as ad
sys.path.append("../resources/scRNA/")
import zc_function as zc

In [ ]:
# make output directory
import os
if not os.path.exists("scRNA_out"):
    os.mkdir("scRNA_out")

if not os.path.exists("scRNA_out/FFPE_scRNA/"):
    os.mkdir("scRNA_out/FFPE_scRNA/")

In [ ]:
def convert_cell_type( dat, orig_obs_col = "celltype_annotation", new_obs_col = "Cell_Type"):
    """Convert cell type annotation string format
    @param dat: the anndata object intended to be modified
    @param orig_obs_col: original obs column where the format wants to be changed
    @param new_obs_col: the new obs column where the formated annotation will be stored
    
    return the anndata object with the new_obs_col added"""
    
    ct_df = dat.obs[orig_obs_col] # cell type dataframe
    ct_df2 = [ct_df[i].split("_")[1] for i in range( len(ct_df) )] #eg  '2_Goblet cells' --> 'Goblet cells'
    dat.obs[new_obs_col ] = ct_df2
    
    return dat

## check signatures in the entire dataset

In [ ]:
dat = sc.read("../data/scRNA/outer_combined_all4_dat.h5ad")

In [ ]:
dat

In [ ]:
#dat.X.sum(axis = 1) #check if is raw

In [ ]:
dat.raw = dat.copy() # save raw layer

In [ ]:
dat.layers['arcsin'] = dat.raw.X.copy() # prep for an arcsin layer

In [ ]:
sc.pp.normalize_total(dat, layer = 'arcsin') # normalize library size

In [ ]:
dat.layers['arcsin'] = np.arcsinh(dat.layers['arcsin'])

### get signatures

In [ ]:
immune_exclusive = ["DDR1", "TGFBI", "PAK4", "DPEP1"]

### make signature scores

In [ ]:
dat.X.sum(axis = 1) #check if data is raw

In [ ]:
dat.raw = dat.copy()

In [ ]:
#dat.raw.X.sum(axis = 1)

In [ ]:
zc.normalization(dat) #normalize library size, arcsin transform and z-score scalling within genes 

In [ ]:
dat.X.sum(axis = 1)

In [ ]:
sc.tl.score_genes(dat, gene_list=immune_exclusive, score_name='IES', use_raw = True) # use raw layer

In [ ]:
sc.tl.score_genes(dat, gene_list=immune_exclusive, score_name='IES_norm', use_raw = False) #use normalized layer

In [ ]:
sc.pl.umap(dat, color = ['Cell_Type', 'SampleId'  ], ncols=1)

In [ ]:
sc.pl.umap(dat, color = immune_exclusive, use_raw = False, ncols=2, vmax = 5) # plot individual gene expression level

In [ ]:
sc.pl.umap(dat, color = ['IES_norm'  , 'IES'], ncols=1, vmax = 2, vmin = 0, s= 2)

examine the distribution of the signature scores (raw and normalized)

In [ ]:
dat.obs["IES_norm"].hist(bins = 100)

In [ ]:
dat.obs["IES"].hist(bins = 100)

make a dictionary where keys are major cell types and values are cell type annotations

In [ ]:
cell_category = dict()
cell_category["Epithelial"] = [ 'Goblet cells', 'Enterocytes','Cholangiocytes','Basal cells','Crypt cells',
                               'Epithelial cells', 'Ductal cells' ] 
cell_category["Immune"] = ['Macrophages', 'T memory cells', 'T cells','Plasma cells',
                           'B cells memory','B cells',  'Mast cells','NK cells', 'Neutrophils',
                           'Plasmacytoid dendritic cells','Dendritic cells','Alveolar macrophages',]
cell_category["Mesenchymal"] = ['Fibroblasts','Pericytes'] 
cell_category["Neuronal"] = ['Enteric glia cells', 'Schwann cells','Enteric neurons',]
cell_category["Endothelial"] = [ 'Endothelial cells',]
cell_category["Other"] = ['Smooth muscle cells', 'Adipocytes',]

reverse the key and values to make new adata obs column

In [ ]:
cell_category_rev = dict()
for k,v in cell_category.items():
    for j in range(len(v)):
        cell_category_rev[v[j]] = k
        

In [ ]:
#cell_category_rev

In [ ]:
#dat.obs.Cell_Type.unique().shape

In [ ]:
dat.obs["Major_cell_type"] = [cell_category_rev[k] for k in dat.obs.Cell_Type]

In [ ]:
dat.obs.Major_cell_type.unique()

## Check immune signatures in T cells

### extract T cells

In [ ]:
dat = sc.read("../data/scRNA/outer_combined_all4_dat.h5ad",) # re-read in the data if needed

In [ ]:
dat.obs.Major_cell_type.unique()

In [ ]:
sc.pl.umap(dat, color = ['Cell_Type','SampleId'], ncols=1)

In [ ]:
dat.obs.Cell_Type.unique()

In [ ]:
tc = dat[np.isin( dat.obs.Cell_Type, ['T memory cells', 'T cells'] ) ] # extract T cells based on cell type annotation 

In [ ]:
tc

In [ ]:
tc.X.sum(axis = 1) # check if raw

In [ ]:
tc.raw = tc.copy()

In [ ]:
zc.normalization(tc)

In [ ]:
sc.pl.umap(tc, color = ['Cell_Type', 'CD8A','SampleId'], ncols=1)

## calculate CD8+ T cell abundance in each sample 

In [ ]:
dat.raw.X.sum(axis = 1)

In [ ]:
zc.normalization(dat)

In [ ]:
#check CD8A expression 
sc.pl.umap(dat, color = ['CD8A', 'SampleId','Cell_Type','CD8T_core9'], use_raw = False, ncols=1, vmin = 0, vmax = 8)

### check histogram

In [ ]:
cd8 = dat.to_df()['CD8A'] # CD8A gene expression per cell

In [ ]:
cd8.shape

In [ ]:
cd8_positive = cd8[cd8>-0.2]
cd8_positive.shape

In [ ]:
cd8_positive.hist(bins = 100 )

In [ ]:
# use otsu
zc.get_otsu_threshold(cd8)

will use 0 as threshold since the otsu threshold is too large based on 

In [ ]:
is_cd8 = [str(cd8[i] > 0 and np.isin(dat.obs.Cell_Type[i], ['T cells', 'T memory cells' ])  ) for i in range(dat.n_obs)]

In [ ]:
len(is_cd8)

In [ ]:
#is_cd8

In [ ]:
dat.obs["is_CD8"] = is_cd8

In [ ]:
sc.pl.umap(dat, color = ['CD8A', 'SampleId','Cell_Type','is_CD8'], use_raw = False, ncols=1, vmin = 0, vmax = 8)

In [ ]:
cd8_df = pd.concat([dat.obs.SampleId, dat.obs.is_CD8], axis = 1)

In [ ]:
cd8_df.head()

In [ ]:
bool_convert_dict = {'True':1, 'False':0}

In [ ]:
cd8_df['is_CD8_int'] = [bool_convert_dict[k] for k in cd8_df["is_CD8"]]

In [ ]:
cd8_df["is_CD8_int"].sum()

In [ ]:
cd8_samples = cd8_df[["is_CD8_int", 'SampleId' ] ].groupby(by = 'SampleId').sum()

In [ ]:
cd8_samples.head()

In [ ]:
cd8_samples["CD8_cell_pct"] = cd8_df[["is_CD8_int", 'SampleId' ] ].groupby(by = 'SampleId').mean()

## save data

In [ ]:
dat.X.sum(axis = 1)

In [ ]:
dat.X = dat.raw.X # SAVE ONLY RAW COUNTS

In [ ]:
dat

In [ ]:
dat.write("../data/scRNA/outer_combined_all4_dat.h5ad", compression = 'gzip')

In [ ]:
tc.X = tc.raw.X

In [ ]:
tc.X.sum(axis = 1)

In [ ]:
tc.write('../data/scRNA/combined_all4_tcell_dat.h5ad', compression = 'gzip' )

In [ ]:
cd8_samples.to_csv("scRNA_out/FFPE_scRNA/cd8_cell_sample_wise.csv", header = True, index = True)